### Imports

In [1]:
import pandas as pd

### Read data from 'xlsx' file

In [2]:
# Read 'xlsx' file as dataframe
df = pd.read_excel('../data/processed/data_cuba.xlsx', index_col=None, sheet_name='base_limpia')
# Create a list with dataframe column names
cols = df.columns

### Create pivot table from dataframe

In [3]:
# Create a pivot table from df
episiotomia = pd.pivot_table(df, values=cols[0], index=cols[27],
                         columns=cols[78], aggfunc='count')

### Clean data

In [4]:
# Convert index to columns and reset index
episiotomia = episiotomia.rename_axis('Hospital').reset_index().rename_axis('', axis='columns')

# Delete column and rename other column
episiotomia = episiotomia.drop('No', 1).rename(columns={'Sí': 'Episiotomía'})

# Remove rows with NaN values on column
episiotomia.dropna(subset=['Episiotomía'], inplace=True)

# Reset index
episiotomia = episiotomia.reset_index(drop=True)

# Regular expressions to clean data typos
episiotomia['Hospital'].replace(r'\n|^\s', '', regex=True, inplace=True)
episiotomia['Hospital'].replace(r'^\s', '', regex=True, inplace=True)

episiotomia.shape

/tmp/ipykernel_15387/3066504590.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  episiotomia = episiotomia.drop('No', 1).rename(columns={'Sí': 'Episiotomía'})


(33, 2)

### Merge two dataframes based on a column

In [5]:
directorio = pd.read_excel(r'../data/processed/directorio_hospitales.xlsx', index_col=None, sheet_name='base_limpia', converters={'Long':str, 'Lat': str})

# Merge dataframes by a column
df_join = pd.merge(episiotomia, directorio, on='Hospital', how='left')

df_join.shape

(33, 4)

In [6]:
# Remove rows with NaN values on column
df_join.dropna(subset=['Long'], inplace=True)
# Reset index
df_join = df_join.reset_index(drop=True)

# Save dataframe as 'xlsx' file
# df_join.to_excel('../data/processed/episiotomia_hospitales.xlsx', index = False)
# df_join.to_json('../data/processed/json/episiotomia_hospitales.json', orient='index', force_ascii=False)

df_join.sample(3)

,Hospital,Episiotomía,Long,Lat
6,Hospital Clínico Quirúrgico Juan Bruno Zayas,1.0,20.0346264574946,-75.79881625
1,Hospital General Municipal 27 de Noviembre,1.0,22.509655349554393,-83.50877625329261
13,Hospital General Docente Iván Portuondo,2.0,22.893216869377824,-82.50498853
